# **MURA DATASET visualization and preprocessing**


this note book for loading MURA dataset v1.1 and doing some visualization
and prepare for learning phase with some preprocessing like data augmentation and datagenerator setting


### contents
...
...


change the folowing cell to code if you faced a problem with keras-preprocessing library 
this library update to be able to use flow_from_dataframe api 

!pip install git+https://github.com/keras-team/keras-preprocessing.git


In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # showing and rendering figures
# io related
#from skimage.io import imread


import os
from glob import glob
# not needed in Kaggle, but required in Jupyter
%matplotlib inline 

In [ ]:
#csv files path
path = '../input/mura-v11/MURA-v1.1'
#csv files names
train_image_paths_csv = "train_image_paths.csv"

## Reading Training data paths 

Here we organize all of the images into one DataFrame with adding descriptive colomns so we can easily filter, view and analyze what is available

using read_csv to read the csv file containing the paths

adding new colomns from the image name 

### Data table colomns
* **image_path** : the absolute path to image 
* **label**      : positive/negative the case classification
* **category**   : the part of body in the image(XR_WRIST,XR_SHOULDER,XR_HAND,XR_FINGER,XR_ELBOW,XR_FOREARM,XR_HUMERUS)
* **patientId**  :the study subfolder name withch refer to specific patient

In [ ]:
train_images_paths = pd.read_csv(os.path.join(path,train_image_paths_csv),dtype=str,header=None)
train_images_paths.columns = ['image_path']



In [ ]:
#(doesnt work) train_images_paths['label'] = df[1 if 'positive' in train_images_paths['image_path'] else 0]
#(dosent work) train_images_paths['label'] =  np.where('positive' in train_images_paths['image_path'], 1, 0)
train_images_paths['label'] = train_images_paths['image_path'].map(lambda x:'positive' if 'positive' in x else 'negative')
    #xsplit('/')[4].split('_')[-1])
# (suggest)train_images_paths['category'] =np.resize [train_images_paths['label'].split('/')[3]]
#(suggest)try doing it using map or apply
train_images_paths['category']  = train_images_paths['image_path'].apply(lambda x: x.split('/')[2])  
train_images_paths['patientId']  = train_images_paths['image_path'].apply(lambda x: x.split('/')[3].replace('patient',''))
train_images_paths.head()

In [ ]:
total_number_of_training_images = np.shape(train_images_paths)[0]
print("total number of images:",total_number_of_training_images )
print ("\n\nnumber of null values", train_images_paths.isnull().sum())
print("\n\nnumber of training images:",np.shape(train_images_paths['image_path'])[0])

#df.groupby('a').count()
#values counts table 
categories_counts = pd.DataFrame(train_images_paths['category'].value_counts())
print ('\n\ncategories:\n',categories_counts )
print('\n\nnumber of patients:',train_images_paths['patientId'].nunique())
print('\n\nnumber of labels:',train_images_paths['label'].nunique())
print ('\n\npositive casses:',len(train_images_paths[train_images_paths['label']=='positive']))
print ('\n\nnegative casses:',len(train_images_paths[train_images_paths['label']=='negative']))

## Load Validation data 
repeat prevois steps for validation data 

validation data has another CSV file containing paths

In [ ]:
valid_image_paths_csv = "valid_image_paths.csv"
valid_data_paths = pd.read_csv(os.path.join(path,valid_image_paths_csv),dtype=str,header=None)
valid_data_paths.columns = ['image_path']
print (valid_data_paths.head(5))


In [ ]:
valid_data_paths['label'] = valid_data_paths['image_path'].map(lambda x:'positive' if 'positive' in x else 'negative')
valid_data_paths['category']  = valid_data_paths['image_path'].apply(lambda x: x.split('/')[2])  
valid_data_paths['dir'] =  valid_data_paths['image_path'].apply(lambda x: x.split('/')[1])
valid_data_paths['patientId']  = valid_data_paths['image_path'].apply(lambda x: x.split('/')[3].replace('patient',''))
valid_data_paths.head()

In [ ]:
print("data_shape:",np.shape(valid_data_paths))
print ("\n\nnumber of null values", valid_data_paths.isnull().sum())
print("\n\nnumber of training images:",np.shape(valid_data_paths['image_path']))

validaton_categories_counts = pd.DataFrame(valid_data_paths['category'].value_counts())
print ('\n\ncategories:\n',validaton_categories_counts)
print('\n\nnumber of patients:',valid_data_paths['patientId'].nunique())
print('\n\nnumber of labels:',valid_data_paths['label'].nunique())
print ('\n\npositive casses:',len(valid_data_paths[valid_data_paths['label']=='positive']))
print ('\n\nnegative casses:',len(valid_data_paths[valid_data_paths['label']=='negative']))



## Build the data generator

> ImageDataGenerator Generate batches of tensor image data with real-time data augmentation.
The data will be looped over (in batches).

### augmentation
- rotation
- width shift
- height shift
- zoom range
- horizontal flip
- vertical flip 

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator
from keras_preprocessing.image import ImageDataGenerator

idg_train_settings = dict(samplewise_center = True,
                         samplewise_std_normalization = True,
                          rotation_range = 5, 
                          width_shift_range = 0.1, 
                         height_shift_range = 0.1,
                         zoom_range = 0.1, 
                         horizontal_flip = True,
                         vertical_flip = True)
idg_train = ImageDataGenerator(**idg_train_settings)

idg_valid_settings = dict(samplewise_center = True,
                         samplewise_std_normalization = True,
                          rotation_range = 0, 
                          width_shift_range = 0., 
                         height_shift_range = 0.,
                         zoom_range = 0.0, 
                         horizontal_flip = False,
                         vertical_flip = False)
idg_valid = ImageDataGenerator(**idg_valid_settings)

choose category

In [ ]:
# put in train_df_of_cat all train_images_paths if you want to train on all dataset
# we will choose category which is not too much images or too many 
# XR_ELBOW or XR_FINGER both have some moderate nomber of examples 
# for train only on XR_ELBOW category we use this mask
category = 'XR_WRIST'
train_mask = train_images_paths['category'] == category
valid_mask = valid_data_paths['category']==category
train_df_of_cat = train_images_paths
valid_df_of_cat = valid_data_paths
train_df_of_cat['label'].hist()

### prepare data for training 
> flow_from_dataframe Takes the dataframe and the path to a directory + generates batches.
The generated batches contain augmented/normalized data.



In [ ]:
import math
images_path_dir = "../input/mura-v11"
out_dir = "./"

train_generator = idg_train.flow_from_dataframe(
    dataframe = train_df_of_cat,
    directory = images_path_dir,
    x_col = 'image_path',
    y_col = 'label',
    batch_size = 32,
    shuffle = True,
    class_mode = "categorical",
    target_size = (128, 128),
    save_to_dir = out_dir,
    save_format = "png",
    color_mode = 'grayscale')

valid_generator = idg_valid.flow_from_dataframe(
    dataframe = valid_df_of_cat,
    directory = images_path_dir,
    x_col = 'image_path',
    y_col = 'label',
    batch_size = 32,
    shuffle = True,
    class_mode = "categorical",
    target_size = (128, 128),
    save_to_dir = out_dir,
    save_format = "png",
    color_mode = 'grayscale')

STEP_SIZE_TRAIN=math.ceil(train_generator.n / train_generator.batch_size)
STEP_SIZE_VALID=math.ceil(valid_generator.n / valid_generator.batch_size)

a, b = next(train_generator)
i,l = next(valid_generator)
print("training input images patch shape  : ", a.shape)
print("training input labels patch shape  : ",b.shape)
print("training labels:",train_generator.class_indices)
print ("________________________________________")
print("validation input images patch shape: ", a.shape)
print("validation input labels patch shape: ",b.shape)
print("validation labels:",valid_generator.class_indices)

# Build a Model
We use an untrained MobileNet to process the images

In [ ]:
input_shape =a.shape[1:]
classes = b.shape[1] #binary classification normal vs upnormal 
loss_func = 'categorical_crossentropy'
optimizer = 'adam'
epochs = 30


In [ ]:
from keras.applications import MobileNet
model = MobileNet(classes=classes,dropout=0.01, weights = None, input_shape=input_shape)
model.compile(optimizer=optimizer, loss=loss_func, metrics = ['acc'])

In [ ]:
print('Layers: {}, parameters: {}'.format(len(model.layers), model.count_params()))

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
file_path=category+"_MobileNet_weights.best.hdf5"
checkpoint = ModelCheckpoint(os.path.join(out_dir,file_path),
                                monitor='val_loss',
                                verbose=1,
                                save_best_only=True,
                                mode='min')
#                              monitor='val_loss', 
#                              verbose=1,
#                              save_best_only=True,
#                              mode='auto')
early = EarlyStopping(monitor="val_acc",
                      mode="max",
                      patience=4)
callbacks_list = [checkpoint,early] 

In [ ]:
%env JOBLIB_TEMP_FOLDER=/tmp

history = model.fit_generator(train_generator, 
                    steps_per_epoch = STEP_SIZE_TRAIN, 
                    validation_data = valid_generator,
                    validation_steps =STEP_SIZE_VALID ,
                    epochs=epochs,
                   callbacks = callbacks_list)

In [ ]:
import math
score = model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)
print("Accuracy:",score[1])
print("Loss    :",score[0])

In [ ]:
from IPython.display import FileLink
FileLink(file_path)


\0
44# Run the validation data
Here we run the validation data with minimal augmentation to see how well it performs and calculate a few stats

In [ ]:
from keras.models import load_model
out_dir = "./"
model.load_weights(os.path.join(out_dir,file_path)) # load the best model

In [ ]:
training_loss_values = history.history['loss']
validation_loss_values = history.history['val_loss']

epochs = range(1, len(training_loss_values)+1)

plt.plot(epochs, training_loss_values, label='Training Loss')
plt.plot(validation_loss_values, label = 'Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
#add test if available
from sklearn.metrics import classification_report, confusion_matrix

valid_generator.reset()
pred=model.predict_generator(valid_generator,steps=STEP_SIZE_validate,verbose=1)
print(pred)
y_true = valid_generator.classes
y_pred =list(np.argmax([pred > 0.5],axis=-1).flatten()) 

print("kk",np.shape(y_true))
print("gg",np.shape(y_pred))
print(train_generator.class_indices)

In [ ]:
print(y_true)


In [ ]:
print(y_pred)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true,y_pred ))
# plt.matshow(confusion_matrix(y_true,y_pred ))

# Make an ROC
Here we create a simple ROC curve to evaluate the model

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
fpr, tpr, thresholds = roc_curve(y_true,y_pred )
fig, ax1 = plt.subplots(1,1)
ax1.plot(fpr, tpr, 'r.', label = 'MobileNet (AUC:%2.2f)' % roc_auc_score(y_pred, y_true))
ax1.plot(fpr, fpr, 'b-', label = 'Random Guessing')
ax1.legend()
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')

In [ ]:
# clean up the virtual directories
import shutil
for c_dir in glob('v_*'):
    shutil.rmtree(c_dir)